In [1]:
import os
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time

# Function to extract URLs from table cells
def url_extract(tds):
    results = []
    for td in tds:
        if td.a:
            results.append(td.a['href'])
        else:
            results.append(None)
    return results

# Define headers for different types of data
draft_header = ['round', 'pick', 'team', 'player', 'pos', 'age', 'to', 'ap1', 'pb', 'st', 'carav', 'drav', 'games', 'pass_cmp', 'pass_att', 'pass_yds', 'pass_tds', 'pass_ints', 'rush_att', 'rush_yds', 'rush_tds', 'receptions', 'rec_yds', 'rec_tds', 'tackles', 'ints', 'sacks', 'college', 'stats']

# Function to parse tables from Pro-Football-Reference
def parse_pfr_tables(tables):
    results = []
    for tbl in tables:
        id_ = tbl.get('id')
        if id_ in headers:
            df = pd.read_html(str(tbl))[0].iloc[1:]
            df.columns = headers[id_]
            results.append(df)
    return pd.concat(results)

# Create directory for caching HTML files
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

def read_html_cache(url, year):
    fn = f"{year}_{url.split('/')[-1]}"
    fn_path = os.path.join(cache_dir, fn)
    if not os.path.exists(fn_path):
        with urlopen(url) as response:
            html = response.read().decode('utf-8')
            with open(fn_path, 'w') as f:
                f.write(html)
    with open(fn_path, 'r') as f:
        return BeautifulSoup(f.read(), 'html.parser')

# Function to extract player URLs from table cells
def player_url_extract(tds):
    results = []
    for td in tds:
        if td.find('a'):
            player_url = td.find('a')['href']
            if player_url.startswith('/players'):
                # Construct full player URL and append to results
                full_url = f"http://www.pro-football-reference.com{player_url}"
                results.append(full_url)
            else:
                results.append(None)
        else:
            results.append(None)
    return results

# Define years to process
years_to_process = range(2000, 2025)

# Scrape draft data
draft_data = []
for year in years_to_process:
    time.sleep(4)
    url = f'http://www.pro-football-reference.com/years/{year}/draft.htm'
    soup = read_html_cache(url, year)
    tables = soup.find_all('table')
    urls = url_extract(tables[0].find_all('td', {'data-stat': 'college_link'}))
    player_urls = player_url_extract(tables[0].find_all('td', {'data-stat': 'player'}))
    draft_table = pd.read_html(str(tables[0]))[0]
    draft_table.columns = draft_header
    draft_table = draft_table[draft_table['pos'] != 'Pos']
    draft_table['college_url'] = urls
    draft_table['player_url'] = player_urls
    # Add a new column 'Year' to the DataFrame
    draft_table['Year'] = [year for _ in range(len(draft_table))]
    draft_data.append(draft_table)

# Concatenate all draft data DataFrames
draft_table = pd.concat(draft_data, ignore_index=True)




C:\Users\benfi\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\benfi\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\benfi\AppData\Local\Temp/ipykernel_27008/3196951228.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  draft_table = pd.read_html(str(tables[0]))[0]
C:\Users\benfi\AppData\Local\Temp/ipykernel_27008/3196951228.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  draft_tab

In [2]:
# Drop the 'combine_stats' column
draft_table.drop('stats', axis=1, inplace=True)

draft_table.head()

,round,pick,team,player,pos,age,to,ap1,pb,st,...,receptions,rec_yds,rec_tds,tackles,ints,sacks,college,college_url,player_url,Year
0,1,1,CLE,Courtney Brown,DE,22,2005,0,0,4,...,0,0,0,156,NaN,19.0,Penn St.,http://www.sports-reference.com/cfb/players/co...,http://www.pro-football-reference.com/players/...,2000
1,1,2,WAS,LaVar Arrington,LB,22,2006,0,3,5,...,0,0,0,338,3,23.5,Penn St.,http://www.sports-reference.com/cfb/players/la...,http://www.pro-football-reference.com/players/...,2000
2,1,3,WAS,Chris Samuels,T,23,2009,0,6,9,...,0,0,0,NaN,NaN,NaN,Alabama,http://www.sports-reference.com/cfb/players/ch...,http://www.pro-football-reference.com/players/...,2000
3,1,4,CIN,Peter Warrick,WR,23,2005,0,0,4,...,275,2991,18,3,NaN,NaN,Florida St.,http://www.sports-reference.com/cfb/players/pe...,http://www.pro-football-reference.com/players/...,2000
4,1,5,BAL,Jamal Lewis,RB,21,2009,1,1,9,...,221,1879,4,NaN,NaN,NaN,Tennessee,http://www.sports-reference.com/cfb/players/ja...,http://www.pro-football-reference.com/players/...,2000


In [3]:
# Replace "http" with "https" in the 'college_url' and 'player_url' columns
draft_table['college_url'] = draft_table['college_url'].str.replace('http://', 'https://')
draft_table['player_url'] = draft_table['player_url'].str.replace('http://', 'https://')

draft_table.head()


,round,pick,team,player,pos,age,to,ap1,pb,st,...,receptions,rec_yds,rec_tds,tackles,ints,sacks,college,college_url,player_url,Year
0,1,1,CLE,Courtney Brown,DE,22,2005,0,0,4,...,0,0,0,156,NaN,19.0,Penn St.,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
1,1,2,WAS,LaVar Arrington,LB,22,2006,0,3,5,...,0,0,0,338,3,23.5,Penn St.,https://www.sports-reference.com/cfb/players/l...,https://www.pro-football-reference.com/players...,2000
2,1,3,WAS,Chris Samuels,T,23,2009,0,6,9,...,0,0,0,NaN,NaN,NaN,Alabama,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
3,1,4,CIN,Peter Warrick,WR,23,2005,0,0,4,...,275,2991,18,3,NaN,NaN,Florida St.,https://www.sports-reference.com/cfb/players/p...,https://www.pro-football-reference.com/players...,2000
4,1,5,BAL,Jamal Lewis,RB,21,2009,1,1,9,...,221,1879,4,NaN,NaN,NaN,Tennessee,https://www.sports-reference.com/cfb/players/j...,https://www.pro-football-reference.com/players...,2000


In [4]:
draft_table.columns = [f"draft_{col}" for col in draft_table.columns]

draft_table.head()

,draft_round,draft_pick,draft_team,draft_player,draft_pos,draft_age,draft_to,draft_ap1,draft_pb,draft_st,...,draft_receptions,draft_rec_yds,draft_rec_tds,draft_tackles,draft_ints,draft_sacks,draft_college,draft_college_url,draft_player_url,draft_Year
0,1,1,CLE,Courtney Brown,DE,22,2005,0,0,4,...,0,0,0,156,NaN,19.0,Penn St.,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
1,1,2,WAS,LaVar Arrington,LB,22,2006,0,3,5,...,0,0,0,338,3,23.5,Penn St.,https://www.sports-reference.com/cfb/players/l...,https://www.pro-football-reference.com/players...,2000
2,1,3,WAS,Chris Samuels,T,23,2009,0,6,9,...,0,0,0,NaN,NaN,NaN,Alabama,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
3,1,4,CIN,Peter Warrick,WR,23,2005,0,0,4,...,275,2991,18,3,NaN,NaN,Florida St.,https://www.sports-reference.com/cfb/players/p...,https://www.pro-football-reference.com/players...,2000
4,1,5,BAL,Jamal Lewis,RB,21,2009,1,1,9,...,221,1879,4,NaN,NaN,NaN,Tennessee,https://www.sports-reference.com/cfb/players/j...,https://www.pro-football-reference.com/players...,2000


In [5]:
# Specify the directory where you want to save the CSV file
data_directory = r"C:\Users\benfi\Defensive_Players_NFL\data"

# Define the path to save the CSV file
csv_path = os.path.join(data_directory, "draft_table.csv")

# Save the DataFrame as a CSV file
draft_table.to_csv(csv_path, index=False)

print("CSV file saved successfully!")

CSV file saved successfully!


In [6]:
draft_table

,draft_round,draft_pick,draft_team,draft_player,draft_pos,draft_age,draft_to,draft_ap1,draft_pb,draft_st,...,draft_receptions,draft_rec_yds,draft_rec_tds,draft_tackles,draft_ints,draft_sacks,draft_college,draft_college_url,draft_player_url,draft_Year
0,1,1,CLE,Courtney Brown,DE,22,2005,0,0,4,...,0,0,0,156,NaN,19.0,Penn St.,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
1,1,2,WAS,LaVar Arrington,LB,22,2006,0,3,5,...,0,0,0,338,3,23.5,Penn St.,https://www.sports-reference.com/cfb/players/l...,https://www.pro-football-reference.com/players...,2000
2,1,3,WAS,Chris Samuels,T,23,2009,0,6,9,...,0,0,0,NaN,NaN,NaN,Alabama,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2000
3,1,4,CIN,Peter Warrick,WR,23,2005,0,0,4,...,275,2991,18,3,NaN,NaN,Florida St.,https://www.sports-reference.com/cfb/players/p...,https://www.pro-football-reference.com/players...,2000
4,1,5,BAL,Jamal Lewis,RB,21,2009,1,1,9,...,221,1879,4,NaN,NaN,NaN,Tennessee,https://www.sports-reference.com/cfb/players/j...,https://www.pro-football-reference.com/players...,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6382,7,253,LAC,Cornelius Johnson,WR,23,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Michigan,https://www.sports-reference.com/cfb/players/c...,https://www.pro-football-reference.com/players...,2024
6383,7,254,LAR,KT Leveston,OL,24,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Kansas St.,https://www.sports-reference.com/cfb/players/k...,https://www.pro-football-reference.com/players...,2024
6384,7,255,GNB,Kalen King,CB,21,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Penn St.,https://www.sports-reference.com/cfb/players/k...,https://www.pro-football-reference.com/players...,2024
6385,7,256,DEN,Nick Gargiulo,OL,24,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,South Carolina,https://www.sports-reference.com/cfb/players/n...,https://www.pro-football-reference.com/players...,2024
